In [114]:
import pandas as pd
import numpy as np

In [195]:
def binomial_transition(n, prob):
    return np.random.binomial(n, prob)

def poisson_transition(n, rate):
    return np.random.poisson(np.nan_to_num(n * rate))

transition_b = lambda n, prob: binomial_transition(n, prob)
transition_p = lambda n, rate: poisson_transition(n, rate)


model_settings            = {}
model_settings["m"]       = 100
model_settings["num_pop"] = 2

N0 = np.array([[1e3], [1.5e3]]) * np.ones((model_settings["num_pop"], model_settings["m"]))
C0 = 0.05 * np.ones((model_settings["num_pop"], model_settings["m"]))


M = np.random.uniform(size=(num_pop, num_pop)) * 100
np.fill_diagonal(M, 0)


A = np.random.uniform(300, 1000, (num_pop, 1))  # admissions
D = np.random.uniform(300, 1000, (num_pop, 1))  # discharges
H = np.random.uniform(1000, 2000, (num_pop, 1)) # hospitalizations

Nmean = np.random.uniform(1, 500, (num_pop, 1))


In [196]:
# parameters
γ = 0.05 * np.ones((model_settings["num_pop"], model_settings["m"]))
β = 0.1  * np.ones((model_settings["num_pop"], model_settings["m"]))
δ = 1/120

In [197]:
m       = model_settings["m"]
num_pop = model_settings["num_pop"]

N       = N0
C       = C0 * N
S       =  np.clip(N - C,0,N)

x0     = np.array([C, a2c, inc]) # [num_state_vars x num_pop x m]

In [202]:

Cout = transition_b(list(np.sum(M, axis=1, keepdims=True)), np.clip(np.nan_to_num(C/N), 0, 1))
Cin  = M.T @ np.clip(np.nan_to_num(C/N), 0, 1)

a2c   = transition_b(list(A), γ)                                  # people admitted colonized.
c2out = transition_b(list(D), np.clip(np.nan_to_num(C/N), 0, 1))  # discharged colonized

λ   = β * C/ Nmean

s2c  = transition_b(list(S), 1-np.exp(-λ)) # new colonized
c2s  = transition_p(C, δ) # decolonizations


C = C + a2c - c2out + s2c + c2s + Cin - Cout
C       = np.clip(C, 0, N)

x = check_state_space(np.concatenate((C, a2c, s2c)))


array([[1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
        1000.],
       [1500., 1500., 1500., 1500., 1500., 1500., 1500., 1500., 1500.,
        1500., 1500., 1500., 1500., 1500., 1500., 1500., 1500., 1500.,
        1500., 1500., 1500., 1500., 1500., 1500., 1500., 1500

# the movement model

M is a matrix of [np, np] and C and N vectors of size [np, m].

Mij = individuals moving for i to j.

1. Colonized ppl moving outside unit $i$. The people moving outside of unit i are $\sum_j M_{ij}$.

    $C_{i}^{out}= \sum_j M_{ij} \frac{C_i}{N_i}$

    In code it will be
    ```
    Cout_i = np.sum(M[i, :]) * C[i, :]/N[i, :]
    ```
    So for putting all the people in a matix it will be
    ```
    Cout = np.sum(M, axis=1, keepdims=True) * C / N
    ```
2. Colonized people moving into unit $i$.
   For a 3 metapop model, the people moving into population $i=1$ will be $M_{21}, M_{31}$
   So the people moving inside colonized are $C_1^{out}=M_{21} * C_2/N_2$ and $C_2^{out}=M_{31} * C_3/N_3$.
    As a matrix multiplication this is $M_{:,1}^T \times C/N$, where $M_{:,1}^T=[M_{11}, M_{21}, M_{31}]$ and $C=[C_1, C_2, C_3]^T$.

   The total people moving inside colonized are $C_i^{out}=\sum_j M_{ji} * C_j/N_j$.

   So, in matrix multiplication (I think - not sure) the people moving into all population can be writted as below, gotta be carefull to make the diagonal in M zero!!! or will add people in the same ward.

   $$C_{in} = M^T \times C$$